In [1]:
import pandas as pd
import numpy as np
import dill

In [2]:
datatotals = pd.read_csv("Totals.csv")

In [3]:
datawildlife = pd.read_csv("WildlifeTotals.csv")

In [4]:
datainsects = pd.read_csv("InsectsTotals.csv")

In [5]:
dataplants = pd.read_csv("PlantsTotals.csv")

In [6]:
dataflight = pd.read_csv("./Noel/IntFlightData.csv")

In [7]:
dataflight['County'] = dataflight['DEST_CITY_NAME'].str.rpartition(',')[0].str.replace(",", " ")

In [8]:
dataflight['State'] = dataflight['DEST_CITY_NAME'].str.rpartition(',')[2]

In [9]:
dataflight1 = dataflight[dataflight['State'].str.contains("FL")]

In [10]:
dataflight2 = dataflight1.replace({'West Palm Beach/Palm Beach': 'PalmBeach', 'Tampa': 'Hillsborough', 'Sanford': 'Seminole', 'St. Petersburg': 'Pinellas', 'St. Augustine': 'StJohns', 'Sarasota/Bradenton': 'Sarasota', 'Pensacola': 'Escambia', 'Panama City': 'Bay', 'Fort Lauderdale': 'Broward', 'Daytona Beach': 'Volusia', 'Cocoa Beach': 'Brevard', 'Fort Pierce': 'StLucie', 'Naples': 'Collier','Fort Myers': 'Lee', 'Jacksonville': 'Duval', 'Miami': 'MiamiDade', 'Orlando': 'Orange'}, regex=True)


In [11]:
dataflight3 = dataflight2.groupby('County', as_index=False)['PASSENGERS'].sum()

In [12]:
dataflight4 = dataflight2.groupby('County', as_index=False)['FREIGHT'].sum()

In [13]:
tourtrade = pd.merge(pd.merge(datatotals,dataflight3,on='County'),dataflight4,on='County')

In [14]:
tourtrade['FLIGHTS'] = tourtrade['FREIGHT'] + tourtrade['PASSENGERS']


In [15]:
invasives = pd.merge(pd.merge(datawildlife,dataplants,on='County'),datainsects,on='County')

In [16]:
dataanimals = pd.read_csv("WildlifeDec.csv")

In [17]:
dataleaf = pd.read_csv("PlantsDec.csv")

In [18]:
databeetle = pd.read_csv("InsectsDec.csv")

In [19]:
databeetle['Countcol'] = 1

In [20]:
dataleaf['Countcol'] = 1

In [21]:
dataanimals['Countcol'] = 1

In [22]:
databeetle.index = databeetle.index.map(str) 

In [23]:
dataleaf.index = dataleaf.index.map(str)

In [24]:
dataanimals.index = dataanimals.index.map(str)

In [25]:
databeetle1 = databeetle.groupby('Longitude', as_index=False)['Countcol'].count()

In [26]:
dataleaf1 = dataleaf.groupby('Longitude', as_index=False)['Countcol'].count()

In [27]:
dataanimals1 = dataanimals.groupby('Longitude', as_index=False)['Countcol'].count()

In [28]:
longs = pd.merge(pd.merge(databeetle1,dataleaf1,on='Longitude'),dataanimals1,on='Longitude')

In [29]:
tourtrade.to_csv('tourtrade')

In [30]:
databeetle1.to_csv('databeetle1')

In [31]:
dataanimals1.to_csv('dataanimals1')

In [32]:
dataleaf1.to_csv('dataleaf1')

In [33]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

In [35]:
# output to notebook
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import HoverTool, BoxSelectTool #For enabling tools
from bokeh.palettes import Spectral4
output_notebook()
power = 50

p = figure(plot_width=800, plot_height=800)
# add square with a size, color, and alpha
p.circle(databeetle1['Longitude'], databeetle1['Countcol'], size=12, color="navy", alpha=0.5)
p.circle(dataleaf1['Longitude'], dataleaf1['Countcol'], size=12, color="red", alpha=0.5)
p.circle(dataanimals1['Longitude'], dataanimals1['Countcol'], size=12, color="green", alpha=0.5)

p.title.text = 'Invasive Species Totals by Longitude'

p.legend.location = "top_left"
p.legend.click_policy="hide"

# show the results
TOOLS = [BoxSelectTool(), HoverTool()]
p.xaxis.axis_label = "Longitude"
p.yaxis.axis_label = "Number of Invasive Species by Type"
show(p)



Loading BokehJS ...

In [139]:
from bokeh.charts import output_file, save

output_file('SpeciesType.html')
save(p)

'/home/vagrant/DataFiles/SpeciesType.html'

In [ ]:
#final bar plot (only works as html output file but not in notebook)
from bokeh.io import show, output_file
from bokeh.plotting import figure

output_file("CountyCountsFinal.html")

counties = ['Bay','Brevard','Broward','Collier','Duval','Escambia','Hillsborough','Lee','MiamiDade','Orange','PalmBeach','Pinellas','Sarasota','Seminole','StJohns','StLucie','Voluisa']

p = figure(x_range=counties, plot_height=250, plot_width= 900, title="Invasive Species Counts",
           toolbar_location=None, tools="")

p.vbar(x=counties, top=[6, 34, 388, 317, 46, 6, 149, 395, 2441, 46, 735, 70, 88, 16, 6, 57, 13], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
show(p)

In [ ]:
#this version of bar graph works only in notebook but plot width is incorrect
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, output_file
import holoviews as hv
hv.extension('bokeh')

In [ ]:
output_file("mixed.html")
data = {'County': tourtrade['County'], 'Total Invasives': tourtrade['Total']}
bars = hv.Bars(data, hv.Dimension('County'), 'Total Invasives')
# bars.hv.bars(x=factors, top=x, width=0.9, alpha=0.5)
bars

In [ ]:
# Pearson R by passengers
sns.set_context("notebook", font_scale=1.1)
sns.set_style("ticks")

# Create scatterplot of dataframe
sns.lmplot('PASSENGERS', # Horizontal axis
           'Insects', # Vertical axis
           data=tourtrade, # Data source
           fit_reg=True, # Don't fix a regression line
           # Set color
           scatter_kws={"marker": "D", # Set marker style
                        "s": 100}) # S marker size

# Set title
plt.title('Tourism and Total Invasives by County (2017)')

# Set x-axis label
plt.xlabel('Total International Flight Passengers')

# Set y-axis label
plt.ylabel('Total Invasives')

In [ ]:
# Pearson R by freight
sns.set_context("notebook", font_scale=1.1)
sns.set_style("ticks")

# Create scatterplot of dataframe
sns.lmplot('FREIGHT', # Horizontal axis
           'Total', # Vertical axis
           data=tourtrade, # Data source
           fit_reg=Tru, # Don't fix a regression line
           # Set color
           scatter_kws={"marker": "D", # Set marker style
                        "s": 100}) # S marker size

# Set title
plt.title('Trade and Total Invasives by County (2017)')

# Set x-axis label
plt.xlabel('Total International Freight')

# Set y-axis label
plt.ylabel('Total Invasives')